In [1]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.24


In [ ]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import tool
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
# The LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

This code demonstrates a task-oriented agent for a healthcare appointment system:

Simulated Data:
DOCTORS_DB contains doctors categorized by specialty.
APPOINTMENT_SLOTS contains available appointment times for each doctor.
This simulates a simple backend database for demonstration.

Tools:
@tool decorators define functions the agent can call:
FindDoctorsTool(specialty) → Returns doctors for a given specialty.
CheckAvailabilityTool(doctor_name) → Returns available slots for a selected doctor.
BookAppointmentTool(details) → Returns confirmation of the appointment.

These tools take inputs dynamically from the agent, making outputs context-aware.

Prompt:

Guides the agent to behave in a task-oriented way:
Only execute the current step requested by the user.
Do not make autonomous decisions or book automatically.
Use the format Thought → Action → Action Input.

Agent Execution:
Uses create_tool_calling_agent to link the LLM with the tools.
AgentExecutor runs the agent with verbose=True (logs all steps) and max_iterations=1 (task-oriented: only one step per user input).

Example Run
Input: "Find dermatologists"

Output: A dynamic list of dermatologists from the simulated database.
Agent performs exactly one task (find doctors) — user must then request the next step (e.g., check availability or book).

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent


In [ ]:
# Simulated Data
DOCTORS_DB = {
    "Dermatology": ["Dr. Smith", "Dr. Kim"],
    "Cardiology": ["Dr. Patel", "Dr. Lee"],
    "Neurology": ["Dr. Jones", "Dr. Wang"]
}

APPOINTMENT_SLOTS = {
    "Dr. Smith": ["Tomorrow 10AM", "Tomorrow 3PM"],
    "Dr. Kim": ["Tomorrow 11AM", "Tomorrow 4PM"],
    "Dr. Patel": ["Jan 21 9AM", "Jan 22 2PM"],
    "Dr. Lee": ["Jan 20 10AM", "Jan 22 1PM"],
    "Dr. Jones": ["Jan 23 11AM", "Jan 24 3PM"],
    "Dr. Wang": ["Jan 21 9AM", "Jan 24 2PM"]
}

In [ ]:
# Define Tools

@tool
def FindDoctorsTool(specialty: str):
    """Return available doctors for the given specialty."""
    doctors = DOCTORS_DB.get(specialty, [])
    if doctors:
        return f"Available {specialty} doctors: {', '.join(doctors)}"
    return f"No doctors found for specialty: {specialty}"

@tool
def CheckAvailabilityTool(doctor_name: str):
    """Return available appointment slots for the selected doctor."""
    slots = APPOINTMENT_SLOTS.get(doctor_name, [])
    if slots:
        return f"Available slots for Dr. {doctor_name}: {', '.join(slots)}"
    return f"No available slots found for Dr. {doctor_name}"

@tool
def BookAppointmentTool(details: str):
    """Book an appointment and return confirmation."""
    return f"Appointment successfully booked for: {details}"

tools = [FindDoctorsTool, CheckAvailabilityTool, BookAppointmentTool]

In [22]:
# Prompt Template

prompt = ChatPromptTemplate.from_template("""
You are a **task-oriented assistant**.

- Only respond to the user's command **step-by-step**.
- Do **not** book automatically or make decisions unless the user explicitly asks.
- Follow this format:

Thought:
Action: <tool-name>
Action Input: <arguments>

If no tool is needed, just answer.

User Request: {input}
{agent_scratchpad}
""")



agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=1  # Task-oriented: only one step at a time
)



In [ ]:
# ----------------------
# Execute Task
# ----------------------

response = executor.invoke({"input": "Find dermatologists"})
print("\n--- Agent Output ---")
print(response["output"])


In production scenario:
| Component          | Currently                   | Production Version                                                          |
| ------------------ | ---------------------------- | --------------------------------------------------------------------------- |
| Doctor Database    | Hardcoded dictionary         | Connected to live hospital/clinic database or EMR system                    |
| Appointment Slots  | Predefined static slots      | Pulled in real-time from scheduling system / API                            |
| Booking Tool       | Returns a fixed confirmation | Calls the hospital’s appointment API to actually book a slot                |
| Error Handling     | Not included                 | Handle conflicts, unavailable slots, canceled appointments, retries         |
| Logging & Auditing | Verbose print statements     | Centralized logging for compliance, monitoring, and traceability            |
| Security           | None                         | User authentication, HIPAA compliance, encryption of sensitive patient data |
| Task Flow          | Manual, step-by-step         | Could be automated in a **goal-oriented agent** for full end-to-end booking |
